In [1]:
import os
os.chdir('..')

In [2]:
from model.network import Converter
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
import os
import pickle
from pathlib import Path
import re

import numpy as np
import pandas as pd
import tensorflow as tf
import lpips_tf

from keras import backend as K
from keras import optimizers, losses, regularizers
from keras.layers import Conv2D, Dense, UpSampling2D, LeakyReLU, Activation
from keras.layers import Layer, Input, Reshape, Lambda, Flatten, Concatenate, Embedding, GaussianNoise
from keras.models import Model, load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from keras.applications import vgg16
from keras_lr_multiplier import LRMultiplier
from keras.models import Sequential
from assets import AssetManager
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
import cv2

from sklearn.utils import shuffle

In [16]:
model_name = 'smallnorb_model_first_phase'
assets = AssetManager('results')
converter = Converter.load( assets.get_model_dir(model_name), include_encoders=False)

def pred_imgs(imgs):
    curr_imgs = np.stack(imgs, axis=0)
    content_codes = converter.content_encoder.predict(curr_imgs)
    class_codes = converter.class_encoder.predict(curr_imgs)
    class_adain_params = converter.class_modulation.predict(class_codes)
    return content_codes, class_adain_params

loading models...


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


vgg arch:
Model: "vgg"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 64, 64, 3)         0         
_________________________________________________________________
vgg_normalization_2 (VggNorm (None, 64, 64, 3)         0         
_________________________________________________________________
model_2 (Model)              [(None, 64, 64, 64), (Non 14714688  
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [17]:
data = np.load(assets.get_preprocess_file_path('smallnorb_strict_class_train'))
imgs, classes, _, n_classes = data['imgs'], data['classes'], data['contents'], data['n_classes']
imgs = imgs.astype(np.float32) / 255.0

In [18]:
train_content = converter.content_embedding.predict(np.arange(imgs.shape[0]))

In [19]:
class_code = converter.class_embedding.predict(classes)
class_adain_params = converter.class_modulation.predict(class_code)

In [20]:
imgs, classes, train_content, class_code, class_adain_params = \
    shuffle(imgs, classes, train_content, class_code, class_adain_params)

In [21]:
class_code.shape, class_adain_params.shape

((38880, 128), (38880, 4, 256, 2))

In [22]:
def get_model(input_dim):
    model = Sequential()
    model.add(Dense(units=256, activation='relu', input_dim=input_dim))
    model.add(Dense(units=n_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [23]:
onehot_enc = OneHotEncoder()
onehot_classes = onehot_enc.fit_transform(classes.reshape(-1,1))
n_classes = onehot_classes.shape[1]

In [24]:
n_epochs=200

model = get_model(train_content.shape[1])
callbacks = [EarlyStopping('val_accuracy', patience=10),]
model.fit(train_content, onehot_classes, epochs=n_epochs, validation_split=0.3, callbacks=callbacks)

Train on 27216 samples, validate on 11664 samples
Epoch 1/200
27216/27216 [==============================] - 3s 94us/step - loss: 3.7572 - accuracy: 0.0481 - val_loss: 3.5484 - val_accuracy: 0.0703
Epoch 2/200
27216/27216 [==============================] - 3s 93us/step - loss: 3.3749 - accuracy: 0.0931 - val_loss: 3.2683 - val_accuracy: 0.1057
Epoch 3/200
27216/27216 [==============================] - 3s 102us/step - loss: 3.1298 - accuracy: 0.1348 - val_loss: 3.0763 - val_accuracy: 0.1399
Epoch 4/200
27216/27216 [==============================] - 2s 91us/step - loss: 2.9394 - accuracy: 0.1746 - val_loss: 2.9381 - val_accuracy: 0.1631
Epoch 5/200
27216/27216 [==============================] - 2s 88us/step - loss: 2.7847 - accuracy: 0.2073 - val_loss: 2.8088 - val_accuracy: 0.1883
Epoch 6/200
27216/27216 [==============================] - 2s 90us/step - loss: 2.6464 - accuracy: 0.2352 - val_loss: 2.6700 - val_accuracy: 0.2311
Epoch 7/200
27216/27216 [==============================] - 3s

27216/27216 [==============================] - 3s 95us/step - loss: 0.9254 - accuracy: 0.6993 - val_loss: 1.2355 - val_accuracy: 0.5830
Epoch 56/200
27216/27216 [==============================] - 3s 99us/step - loss: 0.9156 - accuracy: 0.7016 - val_loss: 1.2286 - val_accuracy: 0.5870
Epoch 57/200
27216/27216 [==============================] - 3s 94us/step - loss: 0.9049 - accuracy: 0.7043 - val_loss: 1.2352 - val_accuracy: 0.5881
Epoch 58/200
27216/27216 [==============================] - 3s 99us/step - loss: 0.8936 - accuracy: 0.7088 - val_loss: 1.2361 - val_accuracy: 0.5872
Epoch 59/200
27216/27216 [==============================] - 3s 96us/step - loss: 0.8835 - accuracy: 0.7118 - val_loss: 1.2279 - val_accuracy: 0.5874
Epoch 60/200
27216/27216 [==============================] - 3s 94us/step - loss: 0.8726 - accuracy: 0.7128 - val_loss: 1.2072 - val_accuracy: 0.5953
Epoch 61/200
27216/27216 [==============================] - 3s 95us/step - loss: 0.8672 - accuracy: 0.7178 - val_loss: 

27216/27216 [==============================] - 3s 122us/step - loss: 0.5870 - accuracy: 0.8046 - val_loss: 1.0714 - val_accuracy: 0.6438
Epoch 110/200
27216/27216 [==============================] - 3s 111us/step - loss: 0.5794 - accuracy: 0.8062 - val_loss: 1.0661 - val_accuracy: 0.6445
Epoch 111/200
27216/27216 [==============================] - 3s 128us/step - loss: 0.5778 - accuracy: 0.8072 - val_loss: 1.0777 - val_accuracy: 0.6407
Epoch 112/200
27216/27216 [==============================] - 3s 115us/step - loss: 0.5749 - accuracy: 0.8090 - val_loss: 1.0638 - val_accuracy: 0.6461
Epoch 113/200
27216/27216 [==============================] - 4s 138us/step - loss: 0.5722 - accuracy: 0.8064 - val_loss: 1.0661 - val_accuracy: 0.6494
Epoch 114/200
27216/27216 [==============================] - 3s 128us/step - loss: 0.5651 - accuracy: 0.8109 - val_loss: 1.0527 - val_accuracy: 0.6519
Epoch 115/200
27216/27216 [==============================] - 3s 109us/step - loss: 0.5702 - accuracy: 0.8097

In [25]:
n_epochs=200

model = get_model(class_code.shape[1])
callbacks = [EarlyStopping('val_accuracy', patience=10),]
model.fit(class_code, onehot_classes, epochs=n_epochs, validation_split=0.3, callbacks=callbacks)

Train on 27216 samples, validate on 11664 samples
Epoch 1/200
27216/27216 [==============================] - 10s 355us/step - loss: 0.3115 - accuracy: 0.9723 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoch 2/200
27216/27216 [==============================] - 5s 190us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 3/200
27216/27216 [==============================] - 4s 141us/step - loss: 7.3251e-04 - accuracy: 1.0000 - val_loss: 4.5946e-04 - val_accuracy: 1.0000
Epoch 4/200
27216/27216 [==============================] - 3s 127us/step - loss: 3.2271e-04 - accuracy: 1.0000 - val_loss: 2.2385e-04 - val_accuracy: 1.0000
Epoch 5/200
27216/27216 [==============================] - 3s 120us/step - loss: 1.6533e-04 - accuracy: 1.0000 - val_loss: 1.2060e-04 - val_accuracy: 1.0000
Epoch 6/200
27216/27216 [==============================] - 3s 119us/step - loss: 9.1532e-05 - accuracy: 1.0000 - val_loss: 6.8676e-05 - val_accuracy: 1.0000
Epoch 7/200
27216/27216